In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#import dataset
df = pd.read_csv('/kaggle/input/titanic-train/titanic_train.csv')
df.head(5)

In [ ]:
df.info()

In [ ]:
#see if any missing value
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False, cmap='viridis')

In [ ]:
#class data details

sns.countplot(df['Survived'])

In [ ]:
df['Survived'].value_counts()

In [ ]:
sns.countplot(df['Survived'], hue = df['Sex'])

In [ ]:
sns.countplot(df['Survived'], hue = df['Pclass'], palette='rainbow')

In [ ]:
#see the age data distribution. drop the null values, dont use KDE

sns.displot(df['Age'].dropna(), kde = False, color = 'green', bins = 40)

In [ ]:
sns.countplot(df['SibSp'])

In [ ]:
df['SibSp'].value_counts()

In [ ]:
df['Fare'].hist(figsize = (10,10), color = 'green')

In [ ]:
sns.distplot(df['Fare'], kde = False, color = 'green')

In [ ]:
df['Fare'].value_counts()

# Data Cleaning

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = 'Pclass', y = 'Age', data = df)

## Age Imputation

In [ ]:
def age_impute(col):
    Age = col[0]
    Pclass = col[1]

    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        elif Pclass == 3:
            return 24
    else:
        return Age
        

In [ ]:
df['Age'] = df[['Age', 'Pclass']].apply(age_impute, axis =1)

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False, cmap='viridis')

In [ ]:
#since Cabin has a huge number of data missing, we will drop the column

ddf = df.drop('Cabin',axis=1,) #axis = 0, then row. axis = 1, then column

In [ ]:
sns.heatmap(ddf.isnull(),yticklabels=False, cmap='viridis')

## Handle Categorical Data with dummy encoding


In [ ]:
embark = pd.get_dummies(ddf['Embarked'], drop_first=True)
sex = pd.get_dummies(ddf['Sex'], drop_first=True)

In [ ]:
#drop the columns that are not required

ddf.drop(['Sex', 'Name', 'Ticket', 'Embarked'], axis = 1, inplace=True)

In [ ]:
ddf.head()

In [ ]:
ddf = pd.concat([ddf, sex, embark], axis =1)

In [ ]:
ddf.head()

In [ ]:
ddf['Survived'].value_counts()

## Removing Class Imbalance by oversampling - SMOTE

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

x = ddf.drop(['Survived'], axis = 1)
y = ddf['Survived']

sm = SMOTE(sampling_strategy='auto')
x_res, y_res = sm.fit_resample(x, y)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

x_train, x_test, y_train, y_test = train_test_split(x_res,y_res, test_size = 0.3, random_state = 101)

logmodel = LogisticRegression()
logmodel.fit(x_train, y_train)

In [ ]:
prediction = logmodel.predict(x_test)

In [ ]:
accuracy = confusion_matrix(y_test, prediction)
accuracy

In [ ]:
accuracy = accuracy_score(y_test, prediction)
accuracy*100